### Imports

In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import random
import sys

from Code.ThirdParty.recoxplainer_master.recoxplainer.config import cfg
from Code.ThirdParty.recoxplainer_master.recoxplainer.data_reader import DataReader
from Code.ThirdParty.recoxplainer_master.recoxplainer.models import ALS
from Code.ThirdParty.recoxplainer_master.recoxplainer.recommender import Recommender
from Code.ThirdParty.recoxplainer_master.recoxplainer.evaluator import Splitter, Evaluator
from Code.ThirdParty.recoxplainer_master.recoxplainer.explain import ALSExplainer

## Prepare data

Import the data:

In [2]:
data = DataReader(**cfg.ml100k)

In [3]:
data.dataset.head()

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Re-arrange users' and items' Ids:

In [4]:
data.make_consecutive_ids_in_dataset()

In [5]:
data.dataset.head()

,userId,itemId,rating,timestamp
0,0,0,3,881250949
1,1,1,3,891717742
2,2,2,1,878887116
3,3,3,2,880606923
4,4,4,1,886397596


Because ALS works on implicit feedback we need to binarize it:

In [6]:
data.binarize(binary_threshold=1)

C:\Users\ADE\miniconda3\envs\recoxplainer\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\ADE\miniconda3\envs\recoxplainer\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [7]:
data.dataset.head()

,userId,itemId,rating,timestamp
0,0,0,3,881250949
1,1,1,3,891717742
2,2,2,1,878887116
3,3,3,2,880606923
4,4,4,1,886397596


Prepare train and test sets:

In [8]:
sp = Splitter()
train, test = sp.split_leave_n_out(data, frac=0.1)

## Model

In [9]:
als = ALS(**cfg.model.als)

In [10]:
als.fit(train)

C:\Users\ADE\miniconda3\envs\recoxplainer\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


  0%|          | 0/10 [00:00<?, ?it/s]

True

## Recommendations

In [11]:
recommender = Recommender(train, als)

In [12]:
recommendations = recommender.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

## Evaluations

In [13]:
eva = Evaluator(test)

In [14]:
eva.cal_hit_ratio(recommendations)

0.22660478955230687

In [15]:
eva.cal_ndcg(recommendations)

0.2427643274570978

## Explain

In [16]:
expl = ALSExplainer(als, recommendations, train)

In [17]:
explanations = expl.explain_recommendations()

Computing explanations:   0%|          | 0/9430 [00:00<?, ?it/s]

### Example

In [26]:
sample_users = random.sample(set(data.dataset.userId),10)
sample_expl = explanations[explanations.userId.isin(sample_users)]

In [27]:
content = pd.read_csv('datasets/ml-100k/u.item', sep='|', encoding = "ISO-8859-1", skiprows=0, engine='python', header=None)
content = content.set_index(0)[[1]]
content.columns = ['movie']

In [29]:
app = JupyterDash(__name__)
#app = Dash(__name__)
app.layout = html.Div([
    html.P("userId:"),
    dcc.Dropdown(
        id='userId',
        value=sample_users[0],
        options=[{'value': x, 'label': x} 
                 for x in sample_users],
        clearable=False
    ),
    html.P("Rank:"),
      dcc.Slider(
        id='rank',
        min=1,
        max=10,
        value=1,
        step=1,
        marks={i:'{}'.format(i) for i in range(1,11)}
    ),
    dcc.Graph(id="pie-chart"),
])

@app.callback(
    Output("pie-chart", "figure"), 
    [Input("userId", "value"), 
     Input("rank", "value")])
def generate_chart(userId, rank):
    expl = sample_expl[(sample_expl['userId'] == userId) & (sample_expl['rank'] == rank)]
    df = pd.DataFrame.from_dict(expl.explanations.iloc[0])
    df['item'] = list(content.loc[train.get_original_item_id(df.item)].movie)
    fig = px.pie(df, values='contribution', names='item', title=content.loc[int(expl.itemId)].movie)
    return fig

app.css.config.serve_locally = True
app.scripts.config.serve_locally = True

# Run app and display result inline in the notebook
app.run_server(mode='inline', debug=True,port=8060)
#app.run_server(mode='external',debug=True,port=8060)

In [ ]:
#!pip install -q dash==1.19.0

In [23]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      1.4.0
ansi2html                    1.6.0
appnope                      0.1.2
apyori                       1.1.2
argon2-cffi                  20.1.0
astunparse                   1.6.3
async-generator              1.10
attrs                        20.3.0
backcall                     0.2.0
bleach                       3.2.3
Brotli                       1.0.9
cachetools                   5.3.0
certifi                      2020.12.5
cffi                         1.14.4
chardet                      4.0.0
click                        7.1.2
colorama                     0.4.6
cycler                       0.10.0
Cython                       0.29.34
dash                         1.19.0
dash-bootstrap-components    1.4.1
dash-core-components         1.15.0
dash-html-components         1.1.2
dash-renderer                1.9.0
dash-table                   4.11.2
decorator                    4.4.2
de